# Part II — Core Django (CRUD Web Apps Done Right)  
## 6. URL Routing and Views (Function-Based and Class-Based)

This chapter is where Django starts feeling like “a real web framework,” because you
learn the two essential skills that everything else builds on:

1. **Routing**: mapping URLs to code correctly and maintainably
2. **Views**: writing request handlers that return the right kind of response (HTML,
   JSON, redirect, errors), safely and cleanly

You will learn both **Function-Based Views (FBVs)** and **Class-Based Views (CBVs)**,
including Django’s generic views (ListView, DetailView, CreateView, etc.) used
widely in industry.

---

## 6.0 Learning Outcomes

By the end of this chapter you should be able to:

- Design URL structures that scale (naming, nesting, namespaces, includes).
- Use `path()` and `re_path()` appropriately; understand routing order and matching.
- Use path converters (`<int:pk>`, `<slug:slug>`, etc.) and build a custom converter.
- Name URLs and use `reverse()` / `{% url %}` everywhere (no hardcoded links).
- Write robust FBVs:
  - method handling (GET/POST) with proper status codes
  - validation of query params
  - HTML + JSON responses
  - redirects
  - errors (404/403/400) correctly
- Use CBVs, understand `as_view()`, and customize:
  - `dispatch()`, `get()`, `post()`
  - `get_queryset()`, `get_context_data()`
- Use generic CBVs (ListView/DetailView) to build real pages quickly.
- Add and test custom error handlers (404/500) at the project level.

---

## 6.1 URL Routing Fundamentals (How Django Chooses a View)

Django routing is configured in **URLconfs**: Python lists named `urlpatterns`
containing `path(...)` or `re_path(...)` entries.

When a request comes in (example: `GET /articles/hello-world/`), Django tries to
match the path against patterns **in order**, first match wins.

### 6.1.1 Why “order matters” is not a small detail

If you write:

```python
urlpatterns = [
    path("articles/<slug:slug>/", views.article_detail),
    path("articles/new/", views.article_new),
]
```

Then the request `/articles/new/` matches the first pattern (slug = "new"), and
Django will call `article_detail` instead of `article_new`.

Correct order:

```python
urlpatterns = [
    path("articles/new/", views.article_new),
    path("articles/<slug:slug>/", views.article_detail),
]
```

**Rule:** put more specific routes before more general “catch” routes.

---

## 6.2 `path()` vs `re_path()` (When to Use Which)

### 6.2.1 `path()` (recommended default)
`path()` uses Django’s simpler converter-based routing.

Example:

```python
from django.urls import path

urlpatterns = [
    path("articles/<int:article_id>/", views.article_detail),
]
```

Advantages:
- readable
- safe and consistent
- covers almost all typical cases

### 6.2.2 `re_path()` (regex routing)
`re_path()` uses regular expressions for advanced matching.

Example:

```python
from django.urls import re_path

urlpatterns = [
    re_path(r"^legacy/(?P<year>\d{4})/$", views.legacy_year),
]
```

Use `re_path()` only when:
- you must match a complex legacy pattern
- you need advanced constraints not possible with converters

Industry advice:
- prefer `path()` for maintainability
- use `re_path()` sparingly (regex routes are harder to read and review)

---

## 6.3 Path Converters (What `<int:pk>` Really Means)

Converters appear inside `<...>` in a `path()` pattern:

```python
path("articles/<int:article_id>/", views.article_detail)
```

Meaning:

- Django matches the URL segment in that position.
- It converts it to the specified type.
- It passes it to your view as a keyword argument.

### 6.3.1 Built-in converters (common ones)

- `<int:x>`: digits only, converted to Python `int`
- `<slug:x>`: letters/numbers/hyphen/underscore, useful for SEO slugs
- `<str:x>`: any non-empty string segment (excluding `/`)
- `<uuid:x>`: UUID format
- `<path:x>`: like str but can include `/` (captures remainder)

Examples:

```python
urlpatterns = [
    path("users/<uuid:user_id>/", views.user_detail),
    path("files/<path:subpath>/", views.file_serve),
]
```

### 6.3.2 What happens if conversion fails?
If the URL doesn’t match the converter, the pattern does not match, and Django
continues to the next pattern. If nothing matches, you get a 404.

Example:

- Pattern: `path("items/<int:item_id>/", ...)`
- URL: `/items/abc/` does not match `<int:...>` → no match → likely 404.

### 6.3.3 Why you should prefer converters over manual parsing
Without converters, you would accept a string and manually validate:

- more code
- more bugs
- inconsistent behavior

Converters shift validation into routing and reduce error cases.

---

## 6.4 Custom Path Converters (Industry-Useful Skill)

Sometimes you want a URL that uses a custom format, like:

- order numbers: `ORD-2026-000123`
- usernames with specific rules
- short IDs

### 6.4.1 Create a custom converter for order IDs

Create `pages/converters.py` (or a common `core/converters.py` later):

```python
import re


class OrderIdConverter:
    regex = r"ORD-\d{4}-\d{6}"

    def to_python(self, value: str) -> str:
        return value

    def to_url(self, value: str) -> str:
        return value
```

Register it in `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path, register_converter

from pages.converters import OrderIdConverter

register_converter(OrderIdConverter, "orderid")

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
]
```

Use it in a URL pattern (in `pages/urls.py`):

```python
from django.urls import path

from . import views

urlpatterns = [
    path("orders/<orderid:order_id>/", views.order_detail, name="order_detail"),
]
```

View (in `pages/views.py`):

```python
from django.http import JsonResponse


def order_detail(request, order_id: str):
    return JsonResponse({"order_id": order_id})
```

Now test:

```bash
curl -i http://127.0.0.1:8000/orders/ORD-2026-000123/
```

Try invalid:

```bash
curl -i http://127.0.0.1:8000/orders/INVALID/
```

Result:
- valid matches and calls view
- invalid doesn’t match route, so 404 (correct behavior)

**Why this matters in real projects:**  
Custom converters reduce duplicate validation logic and keep URLs strongly typed.

---

## 6.5 URL Includes and Namespaces (How Big Django Projects Stay Organized)

As projects grow, you do not want one giant `config/urls.py`.

You split URLs per app and include them.

### 6.5.1 App URLconf pattern (best practice)

In `pages/urls.py`:

```python
from django.urls import path

from . import views

app_name = "pages"

urlpatterns = [
    path("", views.home, name="home"),
    path("healthz/", views.healthz, name="healthz"),
]
```

In `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
]
```

### 6.5.2 Why `app_name` matters
`app_name` enables namespacing so URL names don’t collide.

For example, multiple apps might have a `home` route. Namespacing disambiguates:

- `pages:home`
- `blog:home`
- `dashboard:home`

This becomes critical in large systems.

### 6.5.3 Including with an explicit namespace
If you want a namespace even if the app doesn’t define one (or you want a different
namespace), you can do:

```python
path("pages/", include(("pages.urls", "pages"), namespace="pages"))
```

Industry preference:
- define `app_name` in the app’s `urls.py`
- include normally
- use names like `pages:home` in templates

---

## 6.6 URL Reversal (No Hardcoded Links: Industry Standard Django)

Hardcoding URLs like `"/healthz/"` inside templates and code is fragile:
- if the path changes, your links break
- refactoring becomes risky

Instead, Django encourages “reverse lookup” by route name.

### 6.6.1 `reverse()` in Python
Example:

```python
from django.urls import reverse

url = reverse("pages:healthz")
```

Now `url` becomes `"/healthz/"` (or whatever it is configured to be).

### 6.6.2 `{% url %}` in templates
Example:

```django
<a href="{% url 'pages:home' %}">Home</a>
```

If you rename `/` to `/welcome/`, this still works after updating `urls.py`.

### 6.6.3 URL reversal with parameters
If your pattern is:

```python
path("articles/<int:article_id>/", views.article_detail, name="detail")
```

Then reverse like:

```python
reverse("articles:detail", kwargs={"article_id": 42})
```

In templates:

```django
<a href="{% url 'articles:detail' article_id=42 %}">Read</a>
```

**Why this matters in real teams:**  
Refactors become safe. This is one of Django’s biggest maintainability wins.

---

## 6.7 Views: Function-Based Views (FBVs) Done Properly

A view is a callable:

- input: `request`
- output: `HttpResponse` (or subclass)

### 6.7.1 The request object: what to expect
You commonly use:

- `request.method` (GET/POST/...)
- `request.GET` (query params)
- `request.POST` (form fields)
- `request.body` (raw request body, like JSON)
- `request.headers` (headers)
- `request.user` (authenticated user if auth middleware is enabled)

### 6.7.2 Returning responses: the main response types

#### (A) `HttpResponse` (HTML or plain text)
```python
from django.http import HttpResponse


def hello(request):
    return HttpResponse("Hello")
```

You can set a status:

```python
return HttpResponse("Created", status=201)
```

Set content type explicitly:

```python
return HttpResponse("OK", content_type="text/plain")
```

#### (B) `JsonResponse` (JSON APIs)
```python
from django.http import JsonResponse


def api_status(request):
    return JsonResponse({"status": "ok"})
```

Return error with correct status:

```python
return JsonResponse({"error": "bad input"}, status=400)
```

#### (C) Redirect responses
Use `redirect()` (recommended) instead of manually crafting headers.

```python
from django.shortcuts import redirect


def go_home(request):
    return redirect("pages:home")
```

Redirect to a hardcoded URL is possible but less ideal:

```python
return redirect("/healthz/")
```

#### (D) 404: “not found”
Raise `Http404` or use `get_object_or_404` (later with models).

```python
from django.http import Http404


def must_exist(request):
    raise Http404("Not found")
```

---

## 6.8 Method Handling (GET vs POST) in FBVs

A very common beginner mistake is writing a view that “assumes GET” and then breaks
when someone sends POST (or vice versa). In real life:

- browsers can resubmit forms
- crawlers may hit endpoints unexpectedly
- clients may call wrong methods
- security depends on correct method handling

### 6.8.1 Manual method branching (clear and explicit)
Example: a simple contact form handler (still returning JSON for demo):

```python
from django.http import JsonResponse


def contact(request):
    if request.method == "GET":
        return JsonResponse(
            {
                "info": "Send a POST with name and message",
                "expected_fields": ["name", "message"],
            }
        )

    if request.method == "POST":
        name = request.POST.get("name", "").strip()
        message = request.POST.get("message", "").strip()

        errors = {}
        if not name:
            errors["name"] = "Name is required."
        if len(message) < 10:
            errors["message"] = "Message must be at least 10 characters."

        if errors:
            return JsonResponse({"errors": errors}, status=400)

        return JsonResponse({"status": "received", "name": name})

    return JsonResponse({"error": "Method not allowed"}, status=405)
```

#### Why 405 matters
A 405 tells clients:
- “this endpoint exists”
- “but that method is not allowed”

This is more accurate than returning 404.

### 6.8.2 Use Django’s method decorators (cleaner, standard)
Django provides decorators that automatically enforce method rules.

```python
from django.http import JsonResponse
from django.views.decorators.http import require_GET, require_POST


@require_GET
def only_get(request):
    return JsonResponse({"method": "GET"})


@require_POST
def only_post(request):
    return JsonResponse({"method": "POST"})
```

If a client sends the wrong method, Django returns 405 automatically.

Industry preference:
- use `@require_GET`, `@require_POST`, or `@require_http_methods([...])` for
  correctness and clarity.

---

## 6.9 Query Parameters in Views (Full Explanation + Validation Pattern)

Query parameters arrive in `request.GET`.

Example URL:

```text
/paginate-demo/?page=2&page_size=50
```

- `?` starts query string (everything after it modifies the request)
- `&` separates parameters
- values are strings and must be parsed/validated

### 6.9.1 A reusable parsing pattern (safe, readable)

Create `pages/utils.py`:

```python
def parse_int(value: str, *, name: str, minimum: int | None = None) -> int:
    try:
        parsed = int(value)
    except ValueError as e:
        raise ValueError(f"{name} must be an integer") from e

    if minimum is not None and parsed < minimum:
        raise ValueError(f"{name} must be >= {minimum}")

    return parsed
```

Use it in a view:

```python
from django.http import JsonResponse

from .utils import parse_int


def paginate_demo(request):
    page_raw = request.GET.get("page", "1")
    page_size_raw = request.GET.get("page_size", "20")

    try:
        page = parse_int(page_raw, name="page", minimum=1)
        page_size = parse_int(page_size_raw, name="page_size", minimum=1)
    except ValueError as e:
        return JsonResponse({"error": str(e)}, status=400)

    if page_size > 100:
        page_size = 100

    offset = (page - 1) * page_size

    return JsonResponse(
        {
            "page": page,
            "page_size": page_size,
            "offset": offset,
        }
    )
```

#### Why this approach is industry-friendly
- validation logic is reusable
- error messages are consistent
- you avoid scattering `try/except` parsing everywhere

Later, Django forms/DRF serializers will formalize this idea even more.

---

## 6.10 Class-Based Views (CBVs): What They Are and Why They Exist

CBVs solve a maintainability problem: many web pages share structure.

Example:
- list page
- detail page
- create form
- update form
- delete confirm

All these share patterns:
- fetch data
- validate input
- render template
- redirect on success

CBVs give you a structured class with overridable hooks instead of repeating code.

### 6.10.1 What `as_view()` really does
A CBV is a class, but Django needs a callable. `as_view()` returns a function-like
callable that:

- creates an instance of the view class for the request
- calls `dispatch()`
- `dispatch()` routes to `get()` / `post()` etc depending on method

Minimal example:

```python
from django.http import JsonResponse
from django.views import View


class HelloView(View):
    def get(self, request):
        return JsonResponse({"message": "Hello from CBV"})
```

URL:

```python
from django.urls import path

from .views import HelloView

urlpatterns = [
    path("hello-cbv/", HelloView.as_view(), name="hello_cbv"),
]
```

### 6.10.2 Why `dispatch()` matters
`dispatch()` is where method routing happens and where mixins often hook in (auth,
permissions, etc.).

You typically don’t override `dispatch()` early, but you should know it exists.

---

## 6.11 Generic CBVs (Industry Workhorses)

Django ships generic views to cover very common patterns.

Even before models, we can demonstrate structure.

### 6.11.1 `TemplateView` (render a template with minimal code)
```python
from django.views.generic import TemplateView


class AboutView(TemplateView):
    template_name = "pages/about.html"
```

This view:
- supports GET
- renders the template

Once you learn templates, you’ll use this a lot for static pages.

---

## 6.12 LAB: Build a Real “Articles” Feature (Routes + FBVs + CBVs)

We will create a small “articles” app using **in-memory data** for now (no database
yet). This lets you focus on routing and views without ORM complexity.

### 6.12.1 Create the app
```bash
python manage.py startapp articles
```

Add to `INSTALLED_APPS` in `config/settings.py`:

```python
INSTALLED_APPS = [
    # ...
    "pages",
    "articles",
]
```

### 6.12.2 Create a fake data source
Create `articles/data.py`:

```python
ARTICLES = [
    {
        "id": 1,
        "slug": "hello-django",
        "title": "Hello Django",
        "body": "This is the first article.",
        "tags": ["django", "intro"],
    },
    {
        "id": 2,
        "slug": "routing-and-views",
        "title": "Routing and Views",
        "body": "URLs map to views. Views return responses.",
        "tags": ["django", "routing"],
    },
    {
        "id": 3,
        "slug": "class-based-views",
        "title": "Class-Based Views",
        "body": "CBVs provide structure and reuse.",
        "tags": ["django", "cbv"],
    },
]
```

### 6.12.3 FBV: article list with filtering via query params
Create `articles/views.py`:

```python
from django.http import JsonResponse

from .data import ARTICLES


def article_list(request):
    tag = request.GET.get("tag")
    q = request.GET.get("q")

    results = ARTICLES

    if tag:
        results = [a for a in results if tag in a["tags"]]

    if q:
        q_lower = q.strip().lower()
        results = [
            a
            for a in results
            if q_lower in a["title"].lower() or q_lower in a["body"].lower()
        ]

    return JsonResponse(
        {
            "count": len(results),
            "results": results,
            "filters": {"tag": tag, "q": q},
        }
    )
```

#### Explanation (important details)
- `tag` and `q` are optional query params:
  - `/articles/?tag=django`
  - `/articles/?q=views`
  - `/articles/?tag=django&q=views`
- Query params do not change “what resource” it is (still articles list); they change
  how it’s selected/presented.
- Returning `count` is a common API pattern.

### 6.12.4 FBV: article detail by slug (404 if not found)
Add to `articles/views.py`:

```python
from django.http import Http404, JsonResponse

from .data import ARTICLES


def article_detail(request, slug: str):
    for a in ARTICLES:
        if a["slug"] == slug:
            return JsonResponse(a)

    raise Http404("Article not found")
```

#### Why raise `Http404` instead of returning JSON with 404 manually?
You can do either, but raising `Http404`:
- integrates with Django’s 404 handling system
- uses consistent handling across your project

In APIs, you may prefer explicit JSON 404 bodies later; DRF standardizes this.

### 6.12.5 Add `articles/urls.py` with namespacing
Create `articles/urls.py`:

```python
from django.urls import path

from . import views

app_name = "articles"

urlpatterns = [
    path("", views.article_list, name="list"),
    path("<slug:slug>/", views.article_detail, name="detail"),
]
```

### 6.12.6 Include in project URLs
Edit `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
    path("articles/", include("articles.urls")),
]
```

### 6.12.7 Test with curl
List:

```bash
curl -i http://127.0.0.1:8000/articles/
```

Filter:

```bash
curl -i "http://127.0.0.1:8000/articles/?tag=django"
```

Search:

```bash
curl -i "http://127.0.0.1:8000/articles/?q=views"
```

Detail:

```bash
curl -i http://127.0.0.1:8000/articles/hello-django/
```

404:

```bash
curl -i http://127.0.0.1:8000/articles/does-not-exist/
```

---

## 6.13 CBV Lab: Turn Article Detail into a CBV

Create `articles/cbv.py` (or keep in `views.py`; separating is optional):

```python
from django.http import Http404, JsonResponse
from django.views import View

from .data import ARTICLES


class ArticleDetailView(View):
    def get(self, request, slug: str):
        for a in ARTICLES:
            if a["slug"] == slug:
                return JsonResponse(a)

        raise Http404("Article not found")
```

Update `articles/urls.py`:

```python
from django.urls import path

from . import views
from .cbv import ArticleDetailView

app_name = "articles"

urlpatterns = [
    path("", views.article_list, name="list"),
    path("<slug:slug>/", ArticleDetailView.as_view(), name="detail"),
]
```

#### What you just learned
- CBV routes must use `.as_view()`
- CBVs map HTTP methods to class methods (`get`, `post`, ...)
- The signature still accepts path parameters as arguments

---

## 6.14 Responses, Headers, and Cookies (Professional-Level Basics)

### 6.14.1 Setting headers in a response
Any response can be treated like a dict for headers:

```python
from django.http import JsonResponse


def with_headers(request):
    response = JsonResponse({"ok": True})
    response["X-App-Version"] = "1"
    return response
```

### 6.14.2 Setting a cookie
```python
from django.http import JsonResponse


def set_pref(request):
    response = JsonResponse({"status": "set"})
    response.set_cookie("preferred_language", "en", samesite="Lax")
    return response
```

**Important security note:** Don’t store secrets in cookies unless properly signed
and designed. Sessions are usually better.

---

## 6.15 Error Handling: 404/403/500 and Custom Handlers

### 6.15.1 How Django decides a 404
Django returns a 404 if:
- no URL pattern matches the path, or
- your view raises `Http404`

### 6.15.2 Custom 404 handler (project-level)
Create `config/views.py`:

```python
from django.http import JsonResponse


def handler404(request, exception):
    return JsonResponse(
        {"error": "not_found", "path": request.path},
        status=404,
    )
```

Update `config/urls.py` to point to it (module-level variables):

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
    path("articles/", include("articles.urls")),
]

handler404 = "config.views.handler404"
```

#### Important: Debug mode affects this
- When `DEBUG=True`, Django may show debug pages and may not use your custom handler
  in all situations.
- Custom handlers matter most in production (`DEBUG=False`).

You’ll fully validate custom handlers later when you learn environment settings.

---

## 6.16 Testing URLs and Views (Correctness + Refactor Safety)

Add tests in `articles/tests.py`:

```python
from django.test import TestCase
from django.urls import reverse


class ArticlesTests(TestCase):
    def test_list_works(self):
        url = reverse("articles:list")
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)

        payload = response.json()
        self.assertIn("count", payload)
        self.assertIn("results", payload)

    def test_filter_by_tag(self):
        url = reverse("articles:list")
        response = self.client.get(url, {"tag": "cbv"})
        self.assertEqual(response.status_code, 200)

        payload = response.json()
        self.assertEqual(payload["count"], 1)
        self.assertEqual(payload["results"][0]["slug"], "class-based-views")

    def test_detail_404(self):
        url = reverse("articles:detail", kwargs={"slug": "nope"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 404)
```

#### Why `reverse()` in tests is a big deal
Tests shouldn’t break when you change URL paths. If you hardcode `"/articles/"`,
renaming becomes painful. Using `reverse()` makes tests resilient.

---

## 6.17 Choosing FBV vs CBV (Industry Guidance)

Use **FBVs** when:
- the logic is simple and unique
- you want maximum explicitness
- you’re building small endpoints or custom flows
- your team prefers function style

Use **CBVs** when:
- the view fits a standard CRUD pattern
- you want reusable hooks and mixins
- you want generic views (List/Detail/Create/Update/Delete)
- you expect many similar endpoints

A very common professional approach:
- CBVs for CRUD pages
- FBVs for “special” endpoints (webhooks, custom actions, tiny utilities)

---

## 6.18 Exercises (Do These Before Proceeding)

1. Add an `articles/<int:article_id>/` route alongside slug detail:
   - Create a view that finds by `id`
   - Ensure it returns 404 if missing
   - Add tests using `reverse()`

2. Add method restriction:
   - Make `article_list` GET-only with `@require_GET`
   - Add a test that POST returns 405

3. Add a URL collision intentionally and fix it:
   - Put `<slug:slug>/` above `new/`
   - Observe what breaks
   - Fix ordering and write a short explanation

4. Add namespaced links:
   - In a template (or just in Python), build URLs for:
     - `pages:home`
     - `articles:list`
     - `articles:detail` slug="hello-django"

---

## 6.19 Chapter Summary

- URL routing is ordered matching; “specific before general” prevents bugs.
- `path()` + converters cover most real routing needs; custom converters reduce
  duplicated validation.
- URL naming + `reverse()` is essential for maintainable Django.
- FBVs are explicit and great for custom logic; CBVs provide structure and reuse.
- Correct method handling (405) and error handling (404) are not optional details.
- Tests should use URL reversal to stay stable during refactors.

---

Next chapter: **Part II — Chapter 7: Templates (Django Templating Language)**  
We’ll move from JSON responses to real HTML rendering with template inheritance,
partials, context processors, safe escaping, and building navigation using named URLs.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. Foundations/5. django_architecture_overview.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. templates.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
